<img width=700px; src="../img/logoUPSayPlusCDS_990.png">

<p style="margin-top: 3em; margin-bottom: 2em;"><b><big><big><big><big>Unit testing</big></big></big></big></b></p>

### Outline

1. [Testing principles](#1.-General-testing-principles)
2. [Unit tests](#2.-Unit-tests)
3. [Writing a first test](#3.-Writing-a-test-with-pytest)
4. [Configuring `pytest`](#Pytest-configuration)
5. [Test coverage](#Test-coverage)

#### Foreword

There are several Python libraries dedicated to unit testing. 
The most common are:

* [`unittest`](http://docs.python.org/library/unittest.html)
* [`nose`](http://nosetest.org)
* [`pytest`](http://pytest.org/)

<div class="alert alert-warning">
<b>ATTENTION!</b>: <br><br>
This course focuses on the use of `pytest`.
</div>

## 1. General testing principles

### Why should you write tests ?

In general, tests are an **assessment of** both the **quality** and the **efficiency** of your code.

Tests actually **define** the requirements of the code at various levels. From the basic method definition, to the full software validation.

### What kind of tests should you write ?

For each of these levels, a type of test exists :
- unit tests
- non-regression tests
- pre-integration tests
- integration tests
- validation tests

From this terminology, the unit tests are the basic elements, that should be run **before any commit** of the code. 
They are the one that will be the focus of this course.

## 2. Unit tests

### Definition

A unit test case should 
- test ** individual ** software components ("units") like classes and methods,
- supply mocks or fake versions of dependencies (e.g. database, server, etc.) so that the test does ** not rely on any external object **,
- enable failures to be pinpointed easily.

### Design (theory)

Based on :

    “The Art of Software Testing”, Glenford J. Myers, Corey Sandler, Tom Badgett, Wiley & Sons, 2011

#### Normal conditions

* **Rightness** validate the results against the requirements
  
  *Ex:* a method supposed to select the largest number of a list should be checked by comparing the result with the identified maximum from a known list.
  

* ** Inverse relationships ** 

  *Ex1:* a method calculating the square root of a number can be checked by squaring the result.
  *Ex2:* a method inserting a value in a file can be checked by searching this value after insertion.
  
  This should be done with tools independent from the method to test (other library).


* ** Cross-check: **

  *Ex:* an analytical result can be compared to a numerical calculations for values or conditions where it is possible.


* ** Code logic: **

  *Ex:* 
  ```python
  if ((x or y) and z):
     decision_1
  else: 
     decision_2
  ``` 
  where `x`, `y `and `z` are called "conditions".
  
  Combinations of conditions could be unexpected and lead to a wrong decision. Some decisions are never reached because a different combinations of conditions always lead to the same logical value.

#### Abnormal conditions and edges

* ** Exceptions: **

  If the methods throws exceptions under a certain conditions, this should be checked with a test. See in [section 3](#3.-Writing-a-test-with-pytest).
  
  ```python
  with pytest.raises(<NameOfException>):
      # call to the method that should throw the exception
  ```
  

* ** Boundary conditions: **

  - Does the value exist ?
  - Does the value conform to an expected format ?
  - Is the value given in a reasonable range (min, max) ?
  - Are there enough values (cardinality) ?
  - Does the code reference anything external ?
  - What are the edges of the partitioned values ?
  - Is everything happening in the right order ?


* ** Error conditions: **

  Force error conditions
  - running out of memory
  - running out of disk space
  - issues with wall-clock time
  - network availability and errors
  - system load.


### Test-Driven Development

Test-driven development (TDD) is a software development process, part of the [Agile](https://en.wikipedia.org/wiki/Agile_software_development) principles, that relies on the transcription of the software requirements into tests, before the code that passes the tests is written.

It is an ** iterative process ** that aims at starting with a basic test case andthen ** upgrading alternatively ** the test cases and the code until the requirements are met, depending on the expectations.

## 3. Writing a test with `pytest`

A unit test written under `pytest` is a Python function or class whose name **starts with "test"** and that makes an hypothesis ones considers true.

### A first test

Let's right a basic file containing a function f, and the corresponding test

In [ ]:
%%file my_first_test.py

def f(a):
    return a

def test_a():
    assert f(1) == 1

The file has been saved in the current directory

In [ ]:
!ls *.py

Launching pytest is as easy as move to the right directory and using the command line

    py.test
    
It will start a recursive search from the current directory for Python files, look for methods containing "test" and run them.

In [ ]:
!py.test

For a quick summary, use the quick option `-q`

In [ ]:
!py.test -q

For more information on which test has been run, use the verbose option `-v`

In [ ]:
!py.test -v

The basic test `test_a` has passed.

### Additional tests

Let's now write a bunch of tests, introduce an error on `test_b` and re-run pytest.

In [ ]:
%%file my_second_test.py

def f(a):
    return a

def test_a():
    assert f(1) == 1
    
def test_b():
    assert f(2) == 1

def test_c():
    assert f(3) == 1 + 1 + 1

In [ ]:
!py.test -v

We see pytest has *collected* and run 4 items, 1 from the first file, and 3 from the second. 

As expected, one test has failed.

Therefore `pytest` shows the full traceback leading to the failure, and even gives the output value of the `f` method which can be useful for quick debugging.

### Testing errors and exceptions

The philosophy of Python is to try something first and then decide what to do in case of an error. This is the reason behind Python Exceptions. They inform on the issue that was detected and help the user debug or catch it and find another way to deal with the issue.

When testing a code, it is thus important to assess if these Exceptions are raised as they should be. However, since an exception raised but not caught in an environmment triggers an error, one cannot use the "assert" syntax but the context manager `pytest.raises` instead.


In [ ]:
%%file my_third_test.py

import pytest

def h(n):
    if n < 0:
        raise ValueError("Negative value detected")
    return n
        
def test_h():
    assert h(1) == 1
    
def test_exception_h():
    with pytest.raises(ValueError):
        h(-1)

In [ ]:
!py.test -v my_third_test.py

## Pytest configuration

This part explains how to organize your tests for testing a module. It uses the `euclid` toy module created for this course and available in the base directory under `python-euclid2016/euclid`. 

At this point, it is easier to open a separate terminal, go to the `euclid` directory

    cd ~/Desktop/python-euclid2016/euclid  # for the VM users

and continue from there.

***Remainder***: shell commands in the notebook are preceded with "!", **not** in a terminal.

In [ ]:
# Depending on where you are at this point do not run this
%cd ../euclid/ 

This directory contains a `Makefile` with three utility commands:

    make clean     # to remove __pycache__ and .pyc files
    make tests     # to run the tests
    make coverage  # to run coverage tests (see next section)

In [ ]:
!make clean

---

In order to **visualize** the arborescence of the module and test directory, I recommend using the Linux utility `tree` which can be install in the VM with 

    sudo yum install -y tree

In [ ]:
!tree

Except from the `Makefile`, there are two directories, the module `euclid` and the test directory `tests`.

The `tests` directory contains 

* `__init__.py`  an **empty** file so that the tests are aware of the `euclid` model,
* `test_mytrigo.py`  a file containing the tests for the functions in `mytrigo.py`
* `conftest.py`  a pytest configuration file whose purpose is to **host the fixtures** for all tests

***Note*** `conftest.py` do not need to be imported for pytest to use the fixtures. It is automatic.

---

### Content of `conftest.py`

To visualize the content of the files, one can use

    %load myfile.py


## Test coverage

A test coverage is a report on the number of lignes of a module that have been tested. Basically, a good coverage means much of your code has been run at least once during a test.

To use coverage with pytest, one must first install
    
    sudo pip install pytest-cov

Then the coverage test is run on the module, not on the tests itself. Here the module is `euclid`. before running the coverage test, one needs to be in the directory 

    ../python-euclid2016/euclid

In [ ]:
!py.test --cov euclid/

One can see that `hello.py` and `mytime.py` are not covered by tests.

However, the coverage of `mytime.py` is not 0, as all of the `__init__.py` since the **imports** present in the files trigger an evaluation of some of the lines.